# Pollinator

Does the heavy-lifting for **Pollinations.AI** on Google Colab's free GPUs.

Click *Runtime - Run All*, switch back to the **Pollinations** tab of the browser and wait a little for the connection to be established.

![](https://pollinations.ai/Runtime-RunAll.png?)

In [ ]:
#@title Config and Requirements

!pip install --upgrade pip
!pip install --upgrade typing-extensions==3.10.0.2
!pip install papermill json5
from IPython.display import clear_output
from os.path import join, basename
from shlex import quote

from glob import glob
from json5 import loads

from subprocess import Popen, PIPE, STDOUT
from sys import stdout
import os

#clear_output()
#ipfs_nodeid = generate_slug(2)#asyncio.run(loop.create_task(get_node_id()))
ipfs_root = "/content/ipfs"
worker_root = "/content"
notebook_path = join(ipfs_root,"input", "notebook.ipynb")

print(f"---Settings---\nIPFS_ROOT: {ipfs_root}")

In [ ]:
#@title Empty and create ipfs root: */content/ipfs*



!rm -r $ipfs_root
!mkdir -p $ipfs_root
!mkdir -p $ipfs_root/input
!mkdir -p $ipfs_root/output

clear_output()

In [ ]:
#@title Install Pollinations CLI

%cd /content
!git clone https://github.com/pollinations/pollinations.git
%cd /content/pollinations/app
!git pull
!git checkout dev
!git pull
!npm run install_backend
%cd -

#clear_output()

In [ ]:
#@title Install and Run IPFS daemon &

#!/content/pollinations/ipfs/ipfsd_install_colab.sh
#!nohup /content/pollinations/ipfs/ipfsd_forever.sh &
#!sleep 5
clear_output()


node_id=!tr -dc A-Za-z0-9 </dev/urandom | head -c 13 ; echo ''

node_id=node_id[0]

gpu_description=!nvidia-smi -L
gpu_description=gpu_description[0]


print("---Node ID---:", node_id, gpu_description)

In [ ]:
#@title Create connection to frontend UI

import IPython
from google.colab import output

display(IPython.display.Javascript(f'''
  const iframe = document.createElement("iframe")
  iframe.src = `https://pollinations.ai/incolab.html?node={node_id}&gpu={gpu_description}`
  document.querySelector("#output-area").appendChild(iframe)
  const iframe2 = document.createElement("iframe")
  iframe2.src = `http://localhost:3000/incolab.html?node={node_id}&gpu={gpu_description}`
  document.querySelector("#output-area").appendChild(iframe2)
'''))




In [ ]:
#@title Notebook Runner
from time import sleep
import sys
import matplotlib
matplotlib.use('Agg')
output_path = ipfs_root+"/output"

!pollinate --execute "run_notebook.sh" -l $output_path/log -p $ipfs_root --ipns -n $node_id --debounce 100 > /content/cid
